<a href="https://colab.research.google.com/github/psyuktha/IPC-Sections/blob/main/IPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-03-07 07:40:43--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-03-07 07:40:43 (79.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data,create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
unzip_data("/content/archive (3).zip")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder  # Add this import
from keras.models import Sequential
from keras.layers import Dense, Dropout


file_path = '/content/ipc_sections.csv'
df = pd.read_csv(file_path)

X = df[['Offense', 'Punishment']].astype(str).values  # Convert to string
y = df['Description'].values  # Target variable

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

output_neurons = len(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_text_train = X_train[:, 0] + ' ' + X_train[:, 1]
X_text_test = X_test[:, 0] + ' ' + X_test[:, 1]

# Use CountVectorizer to convert text data to numerical format
vectorizer = CountVectorizer()
X_train_numerical = vectorizer.fit_transform(X_text_train)
X_test_numerical = vectorizer.transform(X_text_test)

sample_weights = compute_sample_weight('balanced', y_train)

model = Sequential()
model.add(Dense(128, input_dim=X_train_numerical.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_neurons, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history_weighted = model.fit(X_train_numerical.toarray(), y_train, epochs=1000,verbose=0, batch_size=100, validation_split=0.2, sample_weight=sample_weights)

test_loss, test_accuracy = model.evaluate(X_test_numerical.toarray(), y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

3/3 [==============================] - 0s 8ms/step - loss: 23.1728 - accuracy: 0.0000e+00
Test Loss: 23.1728
Test Accuracy: 0.0000


In [ ]:
input_text = "theft"

input_numerical = vectorizer.transform([input_text])

prediction = model.predict(input_numerical.toarray())

predicted_label = np.argmax(prediction)

predicted_label_original = label_encoder.inverse_transform([predicted_label])

# Display or use the prediction
print(predicted_label_original)

1/1 [==============================] - 0s 20ms/step
['Description of IPC Section 379\nAccording to section 379 of Indian penal code, Whoever commits theft shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both.\n\n\nIPC 379 in Simple Words\nIn the Indian Penal Code, anyone who commits theft can be punished with imprisonment for up to three years, a fine, or both.']
